# MusicVAE: A Hierarchical Latent Vector Model for Learning Long-Term Structure in Music.
### ___Adam Roberts, Jesse Engel, Colin Raffel, Curtis Hawthorne, and Douglas Eck___

[MusicVAE](https://g.co/magenta/music-vae) learns a latent space of musical scores, providing different modes
of interactive musical creation, including:

* Random sampling from the prior distribution.
* Interpolation between existing sequences.
* Manipulation of existing sequences via attribute vectors.

Examples of these interactions can be generated below, and selections can be heard in our
[YouTube playlist](https://www.youtube.com/playlist?list=PLBUMAYA6kvGU8Cgqh709o5SUvo-zHGTxr).

For short sequences (e.g., 2-bar "loops"), we use a bidirectional LSTM encoder
and LSTM decoder. For longer sequences, we use a novel hierarchical LSTM
decoder, which helps the model learn longer-term structures.

We also model the interdependencies between instruments by training multiple
decoders on the lowest-level embeddings of the hierarchical decoder.

For additional details, check out our [blog post](https://g.co/magenta/music-vae) and [paper](https://goo.gl/magenta/musicvae-paper).
___

This colab notebook is self-contained and should run natively on google cloud. The [code](https://github.com/tensorflow/magenta/tree/master/magenta/models/music_vae) and [checkpoints](http://download.magenta.tensorflow.org/models/music_vae/checkpoints.tar.gz) can be downloaded separately and run locally, which is required if you want to train your own model.

# Basic Instructions

1. Double click on the hidden cells to make them visible, or select "View > Expand Sections" in the menu at the top.
2. Hover over the "`[ ]`" in the top-left corner of each cell and click on the "Play" button to run it, in order.
3. Listen to the generated samples.
4. Make it your own: copy the notebook, modify the code, train your own models, upload your own MIDI, etc.!

# Environment Setup
Includes package installation for sequence synthesis. Will take a few minutes.


In [ ]:
import glob

BASE_DIR = "gs://download.magenta.tensorflow.org/models/music_vae/colab2"

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print('Importing libraries and defining some helper functions...')
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print('Done')

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 1.4MB 7.4MB/s 
     |████████████████████████████████| 92kB 9.3MB/s 
     |████████████████████████████████| 215kB 31.6MB/s 
     |████████████████████████████████| 1.6MB 36.6MB/s 
     |████████████████████████████████| 2.3MB 34.8MB/s 
     |██████████████████████████

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


Instructions for updating:
non-resource variables are not supported in the long term
Done


# 2-Bar Drums Model

Below are 4 pre-trained models to experiment with. The first 3 map the 61 MIDI drum "pitches" to a reduced set of 9 classes (bass, snare, closed hi-hat, open hi-hat, low tom, mid tom, high tom, crash cymbal, ride cymbal) for a simplified but less expressive output space. The last model uses a [NADE](http://homepages.inf.ed.ac.uk/imurray2/pub/11nade/) to represent all possible MIDI drum "pitches".

* **drums_2bar_oh_lokl**: This *low* KL model was trained for more *realistic* sampling. The output is a one-hot encoding of 2^9 combinations of hits. It has a single-layer bidirectional LSTM encoder with 512 nodes in each direction, a 2-layer LSTM decoder with 256 nodes in each layer, and a Z with 256 dimensions. During training it was given 0 free bits, and had a fixed beta value of 0.8. After 300k steps, the final accuracy is 0.73 and KL divergence is 11 bits.
* **drums_2bar_oh_hikl**: This *high* KL model was trained for *better reconstruction and interpolation*. The output is a one-hot encoding of 2^9 combinations of hits. It has a single-layer bidirectional LSTM encoder with 512 nodes in each direction, a 2-layer LSTM decoder with 256 nodes in each layer, and a Z with 256 dimensions. During training it was given 96 free bits and had a fixed beta value of 0.2. It was trained with scheduled sampling with an inverse sigmoid schedule and a rate of 1000. After 300k, steps the final accuracy is 0.97 and KL divergence is 107 bits.
* **drums_2bar_nade_reduced**: This model outputs a multi-label "pianoroll" with 9 classes. It has a single-layer bidirectional LSTM encoder with 512 nodes in each direction, a 2-layer LSTM-NADE decoder with 512 nodes in each layer and 9-dimensional NADE with 128 hidden units, and a Z with 256 dimensions. During training it was given 96 free bits and has a fixed beta value of 0.2. It was trained with scheduled sampling with an inverse sigmoid schedule and a rate of 1000. After 300k steps, the final accuracy is 0.98 and KL divergence is 110 bits.
* **drums_2bar_nade_full**:  The output is a multi-label "pianoroll" with 61 classes. A single-layer bidirectional LSTM encoder with 512 nodes in each direction, a 2-layer LSTM-NADE decoder with 512 nodes in each layer and 61-dimensional NADE with 128 hidden units, and a Z with 256 dimensions. During training it was given 0 free bits and has a fixed beta value of 0.2. It was trained with scheduled sampling with an inverse sigmoid schedule and a rate of 1000. After 300k steps, the final accuracy is 0.90 and KL divergence is 116 bits.

In [ ]:
#@title Load Pretrained Models

drums_models = {}
# One-hot encoded.
drums_config = configs.CONFIG_MAP['cat-drums_2bar_small']
drums_models['drums_2bar_oh_lokl'] = TrainedModel(drums_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/drums_2bar_small.lokl.ckpt')
drums_models['drums_2bar_oh_hikl'] = TrainedModel(drums_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/drums_2bar_small.hikl.ckpt')

# Multi-label NADE.
drums_nade_reduced_config = configs.CONFIG_MAP['nade-drums_2bar_reduced']
drums_models['drums_2bar_nade_reduced'] = TrainedModel(drums_nade_reduced_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/drums_2bar_nade.reduced.ckpt')
drums_nade_full_config = configs.CONFIG_MAP['nade-drums_2bar_full']
drums_models['drums_2bar_nade_full'] = TrainedModel(drums_nade_full_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/drums_2bar_nade.full.ckpt')


INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]

INFO:tensorflow:
Decoder Cells:
  units: [256, 256]

Instructions for updating:
Use `tf.cast` instead.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
INFO:tensorflow:Restoring parameters from gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/drums_2bar_small.lokl.ckpt
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256

# Download & extract midi (1-time only)

In [ ]:
!rm -r audiofiles

os.mkdir('audiofiles')
!wget --no-check-certificate -r "https://ollaregang.pythonanywhere.com/static/MIDI/AllDrums.zip" -O "audiofiles/Input.zip" 

rm: cannot remove 'audiofiles': No such file or directory
will be placed in the single file you specified.

--2021-05-27 09:06:40--  https://ollaregang.pythonanywhere.com/static/MIDI/AllDrums.zip
Resolving ollaregang.pythonanywhere.com (ollaregang.pythonanywhere.com)... 35.173.69.207
Connecting to ollaregang.pythonanywhere.com (ollaregang.pythonanywhere.com)|35.173.69.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6812 (6.7K) [application/zip]
Saving to: ‘audiofiles/Input.zip’

audiofiles/Input.zi 100%[===================>]   6.65K  --.-KB/s    in 0s      

2021-05-27 09:06:40 (615 MB/s) - ‘audiofiles/Input.zip’ saved [6812/6812]

FINISHED --2021-05-27 09:06:40--
Total wall clock time: 0.5s
Downloaded: 1 files, 6.7K in 0s (615 MB/s)


In [ ]:
from zipfile import ZipFile

zipname = "audiofiles/Input.zip"
with ZipFile(zipname,'r') as zpfile:
  files = zpfile.namelist()
  for f in files:
    if (f.endswith('.mid')):
      zpfile.extract(f, 'audiofiles')

# Generate Interpolations

## Load midi files

In [ ]:
input_drums_midi_data = [
    tf.io.gfile.GFile(fn, mode='rb').read()
    for fn in sorted(tf.io.gfile.glob('./audiofiles/MIDI_2bar/*.mid'))]

## convert midi to needed format

In [ ]:
from note_seq.protobuf import music_pb2
import copy
import note_seq

os.mkdir('audiofiles/converted_MIDI')
os.mkdir('audiofiles/ready_to_upload')

drums_input_seqs =music_pb2.NoteSequence();
def set_to_drums(ns):
  for n in ns.notes:
    n.instrument=9
    n.is_drum = True
  return ns


def midi_to_drum_note(input_drums_midi_data):
  drums = []
  for m in input_drums_midi_data:
    drum = mm.midi_to_note_sequence(m)
    set_to_drums(drum)
    drums.append(drum)
  return drums

drums_input_seqs = midi_to_drum_note(input_drums_midi_data)
#print(drums_input_seqs)
#drums_input_seqs = midi_to_note(input_drums_midi_data)
#drums_input_seqs = 
#[mm.midi_to_note_sequence(m) for m in input_drums_midi_data]
print(len(drums_input_seqs))

for index in range(len(drums_input_seqs)):
  # print(seq)
  midi_path = "audiofiles/converted_MIDI/converted_" + str(index) + ".mid"
  mm.note_sequence_to_midi_file(drums_input_seqs[index], midi_path)



10


In [ ]:
input_drums_midi_data = [
    tf.io.gfile.GFile(fn, mode='rb').read()
    for fn in sorted(tf.io.gfile.glob('./audiofiles/converted_MIDI/*.mid'))]

print(input_drums_midi_data)

[b'MThd\x00\x00\x00\x06\x00\x01\x00\x03\x00`MTrk\x00\x00\x00\x1b\x00\xffQ\x03\x07\xa1 \x00\xffX\x04\x04\x02\x18\x08\x00\xffX\x04\x04\x02\x18\x08\x01\xff/\x00MTrk\x00\x00\x00\x07\x00\xc0\x00\x01\xff/\x00MTrk\x00\x00\x00\x86\x00\xc9\x00\x00\x99$c\x00.c\x18$\x00\x00$c\x18$\x00\x00.\x00\x00.c0.\x00\x00.c0.\x00\x00.c0&d\x00.\x00\x00.c0&\x00\x00.\x00\x00.c0.\x00\x00.c0.\x00\x00.c0$d\x00.\x00\x00.c0$\x00\x00.\x00\x00.c0.\x00\x00.c0.\x00\x00.c0&d\x00.\x00\x00.c0&\x00\x00.\x00\x00.c0.\x00\x00.c0.\x00\x00.c0.\x00\x01\xff/\x00', b'MThd\x00\x00\x00\x06\x00\x01\x00\x03\x00`MTrk\x00\x00\x00\x1b\x00\xffQ\x03\x07\xa1 \x00\xffX\x04\x04\x02\x18\x08\x00\xffX\x04\x04\x02\x18\x08\x01\xff/\x00MTrk\x00\x00\x00\x07\x00\xc0\x00\x01\xff/\x00MTrk\x00\x00\x00\xbc\x00\xc9\x00\x00\x99$d\x00.d\x18.\x00\x18.7\x18.\x00\x00.d\x18.\x00\x18.7\x18$\x00\x00$d\x00.\x00\x00.d\x18.\x00\x18$\x00\x00&d\x00.7\x18.\x00\x00.d\x18&\x00\x00.\x000.d\x18.\x00\x18.7\x18.\x00\x00.d\x18.\x00\x00.d\x18.\x00\x18.7\x18.\x00\x00.d\x18$d\x00.

In [ ]:
drums_input_seqs = [mm.midi_to_sequence_proto(m) for m in input_drums_midi_data]

extracted_beats = [];
for ns in drums_input_seqs:

  test=drums_nade_full_config.data_converter.from_tensors(
      drums_nade_full_config.data_converter.to_tensors(ns)[1])
  extracted_beats.extend(test)
  #print(test)
for index, beat in enumerate(extracted_beats):
  print("Beat", index)
  play(beat)



Beat 0


Beat 1


Beat 2


Beat 3


Beat 4


Beat 5


Beat 6


Beat 7


Beat 8


Beat 9


## interpolation

In [ ]:
import requests

alphabet = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]

drums_interp_model = "drums_2bar_oh_hikl" 



# temperature = 0.8 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
temperature = 0.5
num_steps = 8 

for index1, beat1 in enumerate(extracted_beats):
  for index2, beat2 in enumerate(extracted_beats):
    start_beat = extracted_beats[index1]
    end_beat = extracted_beats[index2]
    drums_interp = interpolate(drums_models[drums_interp_model],
                              start_beat, 
                              end_beat, 
                              num_steps=num_steps, 
                              temperature=temperature)
    mm.sequence_proto_to_midi_file(drums_interp, "audiofiles/ready_to_upload/interpolation_" + str(index1) + str(index2) + ".mid")
    url = 'http://OLLAREGANG.pythonanywhere.com/interpolationUploader/interpolation_' + alphabet[index1] + alphabet[index2] + ".mid"
    midiToUpload = open('audiofiles/ready_to_upload/interpolation_' + str(index1) + str(index2) + ".mid", 'rb')
    x = requests.post(url, data = midiToUpload)
    print(x.text)
    # download(drums_interp, "audiofiles/ready_to_upload/interpolation_" + alphabet[index1] + alphabet[index2] + ".mid")

